### Select N-words with higher TF

In [ ]:
import pandas as pd
import sys
import os
import re
sys.path.append(r"C:\Pro\Stages\A4 - DVRC\Work\Supervised-Learning-using-Unsupervised-Learning-Metrics-in-the-absence-of-Annotated-Data\myLibraries")
from MARScore.score import MARSCore
from MARScore.utils import *

In [ ]:
dataset = pd.read_json(r'C:\Pro\Stages\A4 - DVRC\Work\Datasets\pubmed\test.json', lines=True)
dataset = dataset[["article_text", "abstract_text"]]
cleaner = lambda x: ". ".join(x).replace("<S>", "").strip()
format_dot = lambda x: x.replace(" .", ".")
dataset.loc[:,"abstract_text"] = dataset["abstract_text"].replace(regex=r"\[[^\]]*\]", value="")
dataset.loc[:,"article_text"] = dataset["article_text"].replace(regex=r"\[[^\]]*\]", value="")
dataset.loc[:,"abstract_text"] = dataset["abstract_text"].map(cleaner)
dataset.loc[:,"article_text"] = dataset["article_text"].map(cleaner)
dataset.loc[:,"abstract_text"] = dataset["abstract_text"].map(cleanString)
dataset.loc[:,"article_text"] = dataset["article_text"].map(cleanString)
dataset.loc[:,"abstract_text"] = dataset["abstract_text"].map(format_dot)
dataset.loc[:,"article_text"] = dataset["article_text"].map(format_dot)
dataset = dataset.rename(columns={"abstract_text": "summary",
                        "article_text": "text"})

subset = dataset.iloc[3:5, :]
subset

In [ ]:
all_tf = []
for indiv in subset["text"].to_list():
    o, l = tokenizeCorpus(indiv)
    v = vectorizeCorpus(o)
    v, l = cleanAll(v, l)
    all_tf.append(tf(l))

In [ ]:
all_tf

In [ ]:
def best_words_format(tfs, n):
    output = "Maximize\nscore: "
    for i, v in enumerate(tfs.values()):
        output += f"+ {v} c{i}"
    
    output += "\n\nSubject To\n"
    output += "length:"
    for i, v in enumerate(tfs.values()):
        output += f" c{i} +"
    output = output[:-1]
    output += f"< {n}"

    output += "\n\nBinary\n"
    for i, v in enumerate(tfs.values()):
        output += f"c{i}\n"
    return output


In [ ]:
format = best_words_format(all_tf[0], 4)

with open(os.path.join(get_git_root(), r"myLibraries\ilp_outputs\ilp_in.ilp"), "w") as f:
    f.write(format)
    f.close()

os.system(f'glpsol --tmlim 100 --lp "{os.path.join(get_git_root(), r"myLibraries/ilp_outputs/ilp_in.ilp")}" -o "{os.path.join(get_git_root(), r"myLibraries/ilp_outputs/ilp_out.sol")}"')

with open(os.path.join(get_git_root(), r"myLibraries/ilp_outputs/ilp_out.sol"), "r") as f:
    lines = f.readlines()
    f.close()

sentences_lines = [line for line in lines if re.search(r"c\d", line)]

sorted_lines = sorted(sentences_lines, key=lambda line: int(line.split()[1][1:]))
result = [int(sorted_line.split()[3]) for sorted_line in sorted_lines]

In [ ]:
index = [i for i, value in enumerate(result) if value == 1]
index

In [ ]:
ref = []
for i, (k, v) in enumerate(all_tf[0].items()):
    ref.append([i, k, v])
ref.sort(key=lambda x: x[2], reverse=True)
res = [v[0] for v in ref[:4]]
res

### Select N-sentences maximizing words tfs

In [ ]:
import pandas as pd
import sys
import os
import re
sys.path.append(r"C:\Pro\Stages\A4 - DVRC\Work\Supervised-Learning-using-Unsupervised-Learning-Metrics-in-the-absence-of-Annotated-Data\myLibraries")
from MARScore.score import MARSCore
from MARScore.utils import *

In [ ]:
dataset = pd.read_json(r'C:\Pro\Stages\A4 - DVRC\Work\Datasets\pubmed\test.json', lines=True)
dataset = dataset[["article_text", "abstract_text"]]
cleaner = lambda x: ". ".join(x).replace("<S>", "").strip()
format_dot = lambda x: x.replace(" .", ".")
dataset.loc[:,"abstract_text"] = dataset["abstract_text"].replace(regex=r"\[[^\]]*\]", value="")
dataset.loc[:,"article_text"] = dataset["article_text"].replace(regex=r"\[[^\]]*\]", value="")
dataset.loc[:,"abstract_text"] = dataset["abstract_text"].map(cleaner)
dataset.loc[:,"article_text"] = dataset["article_text"].map(cleaner)
dataset.loc[:,"abstract_text"] = dataset["abstract_text"].map(cleanString)
dataset.loc[:,"article_text"] = dataset["article_text"].map(cleanString)
dataset.loc[:,"abstract_text"] = dataset["abstract_text"].map(format_dot)
dataset.loc[:,"article_text"] = dataset["article_text"].map(format_dot)
dataset = dataset.rename(columns={"abstract_text": "summary",
                        "article_text": "text"})

subset = dataset.iloc[3:5, :]
subset

In [ ]:
all_tokens = []
all_tf = []
for indiv in subset["text"].to_list():
    o, l = tokenizeCorpus(indiv)
    v = vectorizeCorpus(o)
    v, l = cleanAll(v, l)
    tfs = tf(l)

    all_tokens.append(l)
    all_tf.append(tfs)

In [ ]:
def best_sentences_format(tokens, tfs, n):
    ### sentence tfs
    i = 0
    sentences_tfs = {}
    for token in tokens:
        if i in sentences_tfs.keys():
            sentences_tfs[i] += tfs[token]
        else:
            sentences_tfs[i] = tfs[token]
        if token == ".":
            i += 1
    
    output = "Maximize\nscore:"
    for i, cur_tf in enumerate(sentences_tfs.values()):
        output += f"+ {cur_tf} s{i}"
    
    output += "\n\nSubject To\n"
    output += f"length:"
    for i in range(len(sentences_tfs.keys())):
        output += f" s{i} +"
    output = output[:-1]
    output += f"< {n}"

    output += "\n\nBinary"
    for i in range(len(sentences_tfs.keys())):
        output += f"\ns{i}"
    return output

In [ ]:
format = best_sentences_format(all_tokens[1], all_tf[1], 4)

with open(os.path.join(get_git_root(), r"myLibraries\ilp_outputs\ilp_in.ilp"), "w") as f:
    f.write(format)
    f.close()

os.system(f'glpsol --tmlim 100 --lp "{os.path.join(get_git_root(), r"myLibraries/ilp_outputs/ilp_in.ilp")}" -o "{os.path.join(get_git_root(), r"myLibraries/ilp_outputs/ilp_out.sol")}"')

with open(os.path.join(get_git_root(), r"myLibraries/ilp_outputs/ilp_out.sol"), "r") as f:
    lines = f.readlines()
    f.close()

sentences_lines = [line for line in lines if re.search(r"s\d", line)]

sorted_lines = sorted(sentences_lines, key=lambda line: int(line.split()[1][1:]))
result = [int(sorted_line.split()[3]) for sorted_line in sorted_lines]

In [ ]:
index = [i for i, value in enumerate(result) if value == 1]
index

In [ ]:
ref = []
### sentence tfs
i = 0
sentences_tfs = {}
for token in all_tokens[1]:
    if i in sentences_tfs.keys():
        sentences_tfs[i] += all_tf[1][token]
    else:
        sentences_tfs[i] = all_tf[1][token]
    if token == ".":
        i += 1


for i, (k, v) in enumerate(sentences_tfs.items()):
    ref.append([i, k, v])
ref.sort(key=lambda x: x[2], reverse=True)
res = [v[0] for v in ref[:4]]
res.sort()
res

### Convex Hull and Inner Cluster Distance

In [ ]:
import pandas as pd
import sys
import os
import re
sys.path.append(r"C:\Pro\Stages\A4 - DVRC\Work\Supervised-Learning-using-Unsupervised-Learning-Metrics-in-the-absence-of-Annotated-Data\myLibraries")
from MARScore.score import MARSCore
from MARScore.utils import *
from hdbscan import HDBSCAN
from umap import UMAP
from scipy.spatial import ConvexHull
from matplotlib import pyplot as plt

In [ ]:
dataset = pd.read_json(r'C:\Pro\Stages\A4 - DVRC\Work\Datasets\pubmed\test.json', lines=True)
dataset = dataset[["article_text", "abstract_text"]]
cleaner = lambda x: ". ".join(x).replace("<S>", "").strip()
format_dot = lambda x: x.replace(" .", ".")
dataset.loc[:,"abstract_text"] = dataset["abstract_text"].replace(regex=r"\[[^\]]*\]", value="")
dataset.loc[:,"article_text"] = dataset["article_text"].replace(regex=r"\[[^\]]*\]", value="")
dataset.loc[:,"abstract_text"] = dataset["abstract_text"].map(cleaner)
dataset.loc[:,"article_text"] = dataset["article_text"].map(cleaner)
dataset.loc[:,"abstract_text"] = dataset["abstract_text"].map(cleanString)
dataset.loc[:,"article_text"] = dataset["article_text"].map(cleanString)
dataset.loc[:,"abstract_text"] = dataset["abstract_text"].map(format_dot)
dataset.loc[:,"article_text"] = dataset["article_text"].map(format_dot)
dataset = dataset.rename(columns={"abstract_text": "summary",
                        "article_text": "text"})

subset = dataset.iloc[3:5, :]
subset

In [11]:
all_embs = []
all_reduced_embs = []
all_tokens = []
all_clabels = []
for indiv in subset["text"].to_list():
    o, l = tokenizeCorpus(indiv)
    v = vectorizeCorpus(o, method="concat_l4")
    v, l = cleanAll(v, l)
    reduced_v, clabels = clusterizeCorpus(UMAP(n_components=2, init="random", random_state=0), v, 
                                        HDBSCAN())
    tf_values = tf(l)
    clusters_tf_values = clusters_tf(tf_values, l, clabels)
    all_embs.append(v)
    all_reduced_embs.append(reduced_v)
    all_tokens.append(l)
    all_clabels.append(clabels)

In [ ]:
def tidy_tokens(tokens, clabels, embs):
    d = {}
    for token, clabel, emb in zip(tokens, clabels, embs):
        if clabel in d.keys():
            d[clabel]["tokens"].append(token)
            d[clabel]["embs"].append(emb)
        else:
            d[clabel] = {"tokens": [token], "embs": [emb]}
    return d

In [ ]:
d_tokens = tidy_tokens(all_tokens[0], all_clabels[0], all_reduced_embs[0])
for k, v in d_tokens.items():
    d_tokens[k]["embs"] = np.array(v["embs"])
convex_hulls = {k: ConvexHull(v["embs"]) for k, v in d_tokens.items()}

In [ ]:
points = d_tokens[0]["embs"]
hull = ConvexHull(points)

# Get the indices of the points forming the convex hull
hull_indices = hull.vertices
hull_indices = np.append(hull_indices, hull.vertices[0])

# Plotting
plt.figure()
plt.plot(points[:, 0], points[:, 1], 'ko', label='Points')
plt.plot(points[hull_indices, 0], points[hull_indices, 1], 'r-', lw=2, label='Convex Hull')
for x, y, index in zip(points[hull_indices, 0], points[hull_indices, 1], hull_indices):
    plt.text(x, y, str(index))
plt.xlabel('X')
plt.ylabel('Y')
plt.title('Convex Hull')
plt.legend()
plt.show()

In [ ]:
def euclideanDistance(p, q):
    p = np.array(p)
    q = np.array(q)
    return np.linalg.norm(p - q)
   

def crossProduct(p, q, r):
    pq = np.array(q) - np.array(p)
    pr = np.array(r) - np.array(p)
    return np.cross(pq, pr)
    

def rotatingCaliper(points, convex_hull):
   
    # Takes O(n)
    hull = [points[i] for i in range(len(points)) if i in convex_hull.vertices]
    n = len(hull)
 
    # Base Cases
    if n == 1:
        return 0
    if n == 2:
        return euclideanDistance(hull[0], hull[1])
    k = 1
 
    # Find the farthest vertex
    # from hull[0] and hull[n-1]
    while crossProduct(hull[n - 1], hull[0], hull[(k + 1) % n]) > crossProduct(hull[n - 1], hull[0], hull[k]):
        k += 1
 
    res = 0
 
    # Check points from 0 to k
    for i in range(k + 1):
        j = (i + 1) % n
        while crossProduct(hull[i], hull[(i + 1) % n], hull[(j + 1) % n]) > crossProduct(hull[i], hull[(i + 1) % n], hull[j]):
            # Update res
            res = max(res, euclideanDistance(hull[i], hull[(j + 1) % n]))
            j = (j + 1) % n
 
    # Return the result distance
    return res

def biggerDistance(points):
    points = np.array(points)
    convex_hull = ConvexHull(points)
    return rotatingCaliper(points, convex_hull)

# Code inspired by amit_mangal_ on geeksforgeeks forum

In [ ]:
rotatingCaliper(d_tokens[0]["embs"], convex_hulls[0]) #complexity = O(N*log(N))

In [ ]:
biggerDistance(d_tokens[0]["embs"])

### Optimized Score

In [1]:
import pandas as pd
import sys
import os
import re
sys.path.append(r"C:\Pro\Stages\A4 - DVRC\Work\Supervised-Learning-using-Unsupervised-Learning-Metrics-in-the-absence-of-Annotated-Data\myLibraries")
from MARScore.score import MARSCore
from MARScore.utils import *
from hdbscan import HDBSCAN
from umap import UMAP
from scipy.spatial import ConvexHull
from matplotlib import pyplot as plt

c:\Pro\Stages\A4 - DVRC\Work\Supervised-Learning-using-Unsupervised-Learning-Metrics-in-the-absence-of-Annotated-Data\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Pro\Stages\A4 - DVRC\Work\Supervised-Learning-using-Unsupervised-Learning-Metrics-in-the-absence-of-Annotated-Data\.venv\Lib\site-packages\umap\distances.py:1063: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @numba.jit()
c:\Pro\Stages\A4 - DVRC\Work\Supervised-Learning-using-Unsupervised-Learning-Metrics-in-the-absence-of-Annotat

In [2]:
dataset = pd.read_json(r'C:\Pro\Stages\A4 - DVRC\Work\Datasets\pubmed\test.json', lines=True)
dataset = dataset[["article_text", "abstract_text"]]
cleaner = lambda x: ". ".join(x).replace("<S>", "").strip()
format_dot = lambda x: x.replace(" .", ".")
dataset.loc[:,"abstract_text"] = dataset["abstract_text"].replace(regex=r"\[[^\]]*\]", value="")
dataset.loc[:,"article_text"] = dataset["article_text"].replace(regex=r"\[[^\]]*\]", value="")
dataset.loc[:,"abstract_text"] = dataset["abstract_text"].map(cleaner)
dataset.loc[:,"article_text"] = dataset["article_text"].map(cleaner)
dataset.loc[:,"abstract_text"] = dataset["abstract_text"].map(cleanString)
dataset.loc[:,"article_text"] = dataset["article_text"].map(cleanString)
dataset.loc[:,"abstract_text"] = dataset["abstract_text"].map(format_dot)
dataset.loc[:,"article_text"] = dataset["article_text"].map(format_dot)
dataset = dataset.rename(columns={"abstract_text": "summary",
                        "article_text": "text"})

subset = dataset.iloc[3:5, :]
subset

,text,summary
3,congenital adrenal hyperplasia ( cah ) refers ...,congenital adrenal hyperplasia is a group of a...
4,type 1 diabetes ( t1d ) results from the destr...,objective(s):pentoxifylline is an immunomodula...


In [3]:
all_embs = []
all_reduced_embs = []
all_tokens = []
all_clabels = []
for indiv in subset["text"].to_list():
    o, l = tokenizeCorpus(indiv)
    v = vectorizeCorpus(o, method="concat_l4")
    v, l = cleanAll(v, l)
    reduced_v, clabels = clusterizeCorpus(UMAP(n_components=2, init="random", random_state=0), v, 
                                        HDBSCAN())
    tf_values = tf(l)
    clusters_tf_values = clusters_tf(tf_values, l, clabels)
    all_embs.append(v)
    all_reduced_embs.append(reduced_v)
    all_tokens.append(l)
    all_clabels.append(clabels)

In [4]:
def tidy_tokens(tokens, clabels, embs):
    d = {}
    for token, clabel, emb in zip(tokens, clabels, embs):
        if clabel in d.keys():
            d[clabel]["tokens"].append(token)
            d[clabel]["embs"].append(emb)
        else:
            d[clabel] = {"tokens": [token], "embs": [emb]}
    return d

In [10]:
d_tokens = tidy_tokens(all_tokens[1], all_clabels[1], all_reduced_embs[1])
for k, v in d_tokens.items():
    d_tokens[k]["embs"] = np.array(v["embs"])

In [11]:
def euclideanDistance(p, q):
    p = np.array(p)
    q = np.array(q)
    return np.linalg.norm(p - q)


def crossProduct(p, q, r):
    pq = np.array(q) - np.array(p)
    pr = np.array(r) - np.array(p)
    return np.cross(pq, pr)


def rotatingCaliper(points, convex_hull):
   
    # Takes O(n)
    hull = [points[i] for i in range(len(points)) if i in convex_hull.vertices]
    n = len(hull)
 
    # Base Cases
    if n == 1:
        return 0
    if n == 2:
        return euclideanDistance(hull[0], hull[1])
    k = 1
 
    # Find the farthest vertex
    # from hull[0] and hull[n-1]
    while crossProduct(hull[n - 1], hull[0], hull[(k + 1) % n]) > crossProduct(hull[n - 1], hull[0], hull[k]):
        k += 1
 
    res = 0
 
    # Check points from 0 to k
    for i in range(k + 1):
        j = (i + 1) % n
        while crossProduct(hull[i], hull[(i + 1) % n], hull[(j + 1) % n]) > crossProduct(hull[i], hull[(i + 1) % n], hull[j]):
            # Update res
            res = max(res, euclideanDistance(hull[i], hull[(j + 1) % n]))
            j = (j + 1) % n
 
    # Return the result distance
    return res

def biggerDistance(points):
    points = np.array(points)
    convex_hull = ConvexHull(points)
    return rotatingCaliper(points, convex_hull)

# Code inspired by amit_mangal_ on geeksforgeeks forum

In [12]:
def optimized_score(tokens_dict, clusters_tfs):
    scores = {}
    for k, v in tokens_dict.items():
        try:
            max_dist = biggerDistance(v["embs"])
            scores[k] = clusters_tfs[k]/(max_dist*len(v["tokens"])) if max_dist > 0 else 0
        except:
            scores[k]=0
    return scores

In [15]:
c_tfs = clusters_tf(tf(all_tokens[1]), all_tokens[1], all_clabels[1])

In [16]:
c_tfs

{82: 534.0,
 121: 3201.0,
 54: 2814.0,
 78: 412.0,
 153: 121.0,
 -1: 7047.0,
 46: 503.0,
 154: 31.0,
 79: 17.0,
 147: 107.0,
 134: 239.0,
 88: 158.0,
 115: 65.0,
 21: 72.0,
 25: 81.0,
 26: 108.0,
 19: 81.0,
 103: 414.0,
 44: 251.0,
 149: 198.0,
 126: 143.0,
 47: 7887.0,
 72: 287.0,
 158: 91.0,
 151: 410.0,
 162: 224.0,
 136: 102.0,
 148: 72.0,
 150: 93.0,
 159: 91.0,
 138: 116.0,
 118: 33.0,
 39: 252.0,
 135: 285.0,
 12: 390.0,
 102: 7.0,
 111: 91.0,
 146: 80.0,
 140: 145.0,
 164: 29.0,
 163: 50.0,
 67: 16.0,
 131: 54.0,
 45: 126.0,
 119: 160.0,
 27: 187.0,
 87: 36.0,
 59: 131.0,
 98: 304.0,
 73: 50.0,
 55: 1072.0,
 63: 530.0,
 7: 143.0,
 8: 218.0,
 1: 154.0,
 16: 147.0,
 10: 143.0,
 56: 350.0,
 145: 35.0,
 160: 492.0,
 165: 56.0,
 110: 280.0,
 51: 22.0,
 107: 16.0,
 117: 457.0,
 41: 323.0,
 18: 359.0,
 30: 883.0,
 42: 71.0,
 58: 135.0,
 124: 74.0,
 40: 936.0,
 71: 81.0,
 81: 162.0,
 65: 56.0,
 38: 56.0,
 24: 105.0,
 61: 121.0,
 62: 99.0,
 53: 73.0,
 85: 251.0,
 92: 1762.0,
 143: 283.0

In [17]:
d_tokens

{82: {'tokens': ['type',
   'diabetes',
   '##1',
   '##d',
   'results',
   'from',
   'destruction',
   'insulin',
   'producing',
   'pan',
   '##cre',
   '##atic',
   'cells',
   'cell',
   'specific',
   'auto',
   '##im',
   '##mun',
   '##e',
   'process',
   'chronic',
   'pan',
   '##cre',
   '##atic',
   'inflammation',
   'ins',
   '##uli',
   '##tis',
   'and',
   'destruction',
   'isle',
   '##t',
   'cells',
   'type',
   'diabetes',
   'mediated',
   'type',
   'diabetes',
   'pathogen',
   '##ic',
   'mechanisms',
   '##pus',
   '##ythe',
   '##mat',
   '##os',
   '##us',
   '##e',
   '##ten',
   'inflammatory',
   'cell',
   'responses',
   'type'],
  'embs': array([[1.354691  , 5.121231  ],
         [1.268636  , 5.1460686 ],
         [1.230098  , 5.139387  ],
         [1.261711  , 5.1655293 ],
         [0.8911067 , 5.225374  ],
         [0.92512375, 5.278199  ],
         [0.95579964, 5.2863574 ],
         [1.1513762 , 5.1435    ],
         [1.0680073 , 5.130753  ],
 

In [14]:
optimized_score(d_tokens, c_tfs)

{82: 0,
 121: 252.99409446202395,
 54: 520.9817463218787,
 78: 73.67253936238083,
 153: 107.56429776259358,
 -1: 4.9214190301694,
 46: 23.371296840135972,
 154: 25.63985403494408,
 79: 6.235203052542828,
 147: 28.263387679234608,
 134: 60.29300651702029,
 88: 246.75050328308333,
 115: 10.302458752929974,
 21: 264.71790481034617,
 25: 259.79532733425475,
 26: 41.63589719090981,
 19: 269.7715746600524,
 103: 25.486783497062596,
 44: 21.29757521718107,
 149: 26.792687342000928,
 126: 0,
 47: 77.87142776464563,
 72: 0,
 158: 100.63731608978482,
 151: 414.5765366450074,
 162: 0,
 136: 83.0082353588312,
 148: 24.07443686464583,
 150: 18.501816577753885,
 159: 57.03227824153284,
 138: 0,
 118: 0,
 39: 575.9894945153402,
 135: 25.08894480474084,
 12: 411.24804673551637,
 102: 6.8783713020184996,
 111: 166.89598815755517,
 146: 32.47671298354557,
 140: 0,
 164: 17.572105333109892,
 163: 18.738344948641938,
 67: 4.246814870526829,
 131: 0,
 45: 328.1416781283377,
 119: 172.50612704657686,
 27: 4

In [12]:
d_tokens[0]["embs"].shape

(32, 2)

In [20]:
for v in list(d_tokens.values())[:1]:
    print(v["embs"].shape)

(52, 2)
